In [ ]:
!rm -rf gym-duckietown/
branch = "daffy" #@param ['master', 'daffy']
! git clone --branch {branch} https://github.com/duckietown/gym-duckietown
!pip install  gym-duckietown/.

Cloning into 'gym-duckietown'...
remote: Enumerating objects: 6116, done.
remote: Counting objects: 100% (397/397), done.
remote: Compressing objects: 100% (202/202), done.
remote: Total 6116 (delta 193), reused 314 (delta 137), pack-reused 5719
Receiving objects: 100% (6116/6116), 79.15 MiB | 22.43 MiB/s, done.
Resolving deltas: 100% (3502/3502), done.
Processing ./gym-duckietown
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for duckietown-gym-daffy: filename=duckietown_gym_daffy-6.1.31-py3-none-any.whl size=129723 sha256=63e5d47e219bcf6a7443693d85893662cadc25714292282b5cb743b108f42a79
  Stored in directory: /root/.cache/pip/

In [ ]:
!pip install pyvirtualdisplay
!pip install piglet
!apt-get install python-opengl -y
!apt install xvfb -y
!apt-get install x11-utils
!pip install torch==1.7.0
!pip install torchvision==0.8.1
!pip install -U "ray[tune]"  
!pip install -U "ray[rllib]"  
!pip install -U "ray[serve]"
!pip install gym==0.22.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-opengl is already the newest version (3.1.0+dfsg-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
xvfb is already the newest version (2:1.19.6-1ubuntu4.10).
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
x11-utils is already the newest version (7.7+3build1).
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'apt autoremove' to remove it.
0 upg

In [ ]:
from pyvirtualdisplay import Display
import matplotlib.pyplot as plt
%matplotlib inline

import os
def create_display():
    display = Display(visible=False, size=(1400, 900))
    d = display.start()
    if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
        !bash ../xvfb start
        %env DISPLAY=:0
    return d

display = create_display()



In [ ]:
import sys
import ray
from ray import tune
import gym_duckietown
from gym_duckietown.envs import DuckietownEnv
from gym_duckietown.simulator import Simulator, DEFAULT_ROBOT_SPEED, DEFAULT_CAMERA_WIDTH, DEFAULT_CAMERA_HEIGHT, CAMERA_FOV_Y, NotInLane
import gym

import pyglet
from collections import deque
from torch import nn
from torch import optim
import numpy as np
import random
from ray.tune import register_env
from ray.rllib.agents.ppo import ppo
from ray.rllib.agents.ppo import PPOTrainer
import random
import yaml
import glob
from ray.tune.logger import pretty_print
from pprint import pprint
import cv2
import logging
from gym import spaces
from matplotlib import pyplot as plt
import seaborn
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import norm

DEBUG:commons:version: 6.2.4 *
DEBUG:typing:version: 6.2.3
DEBUG:duckietown_world:duckietown-world version 6.2.38 path /usr/local/lib/python3.7/dist-packages
DEBUG:geometry:PyGeometry-z6 version 2.1.4 path /usr/local/lib/python3.7/dist-packages
DEBUG:aido_schemas:aido-protocols version 6.0.59 path /usr/local/lib/python3.7/dist-packages
DEBUG:nodes:version 6.2.13 path /usr/local/lib/python3.7/dist-packages pyparsing 3.0.7
DEBUG:gym-duckietown:gym-duckietown version 6.1.31 path /usr/local/lib/python3.7/dist-packages



{'audio': ('directsound', 'openal', 'pulse', 'silent'), 'debug_font': False, 'debug_gl': True, 'debug_gl_trace': False, 'debug_gl_trace_args': False, 'debug_graphics_batch': False, 'debug_lib': False, 'debug_media': False, 'debug_texture': False, 'debug_trace': False, 'debug_trace_args': False, 'debug_trace_depth': 1, 'debug_trace_flush': True, 'debug_win32': False, 'debug_x11': False, 'graphics_vbo': True, 'shadow_window': True, 'vsync': None, 'xsync': True, 'xlib_fullscreen_override_redirect': False, 'darwin_cocoa': True, 'search_local_libs': True}


In [ ]:
logger = logging.getLogger(__name__)


def load_config(path: str = "./main.yaml"):
    # create base config
    config: dict = dict()
    with open(path) as file:
        config = yaml.load(file, yaml.Loader)
    # add sub-configs
    return config


def get_rllib_config(path):
    conf = load_config(path)["rllib_config"]
    conf["env_config"] = env_config(path)
    return conf


def env_config(path):
    conf = load_config(path)
    return conf["env_config"]


def get_default_rllib_conf():
    return ppo.DEFAULT_CONFIG.copy()


def update_conf(conf):
    ray_conf = ppo.DEFAULT_CONFIG.copy()
    for key in conf:
        if key in ray_conf:
            ray_conf[key] = conf[key]
    return ray_conf

In [ ]:
class Heading2WheelVelsWrapper(gym.ActionWrapper):
    def __init__(self, env, heading_type=None):
        super(Heading2WheelVelsWrapper, self).__init__(env)
        self.heading_type = heading_type
        self.action_space = spaces.Box(low=-1., high=1., shape=(1,))
        if self.heading_type == 'heading_trapz':
            straight_plateau_half_width = 0.3333  # equal interval for left, right turning and straight
            self.mul = 1. / (1. - straight_plateau_half_width)

    def action(self, action):
        if isinstance(action, tuple):
            action = action[0]
        # action = [-0.5 * action + 0.5, 0.5 * action + 0.5]
        if self.heading_type == 'heading_smooth':
            action = np.clip(np.array([1 + action ** 3, 1 - action ** 3]), 0., 1.)  # Full speed single value control
        elif self.heading_type == 'heading_trapz':
            action = np.clip(np.array([1 - action, 1 + action]) * self.mul, 0., 1.)
        elif self.heading_type == 'heading_sine':
            action = np.clip([1 - np.sin(action * np.pi), 1 + np.sin(action * np.pi)], 0., 1.)
        elif self.heading_type == 'heading_limited':
            action = np.clip(np.array([1 + action*0.666666, 1 - action*0.666666]), 0., 1.)
        else:
            action = np.clip(np.array([1 + action, 1 - action]), 0., 1.)  # Full speed single value control
        return action

In [ ]:
class ClipImageWrapper(gym.ObservationWrapper):
    def __init__(self, env=None, top_margin_divider=3):
        super(ClipImageWrapper, self).__init__(env)
        img_height, img_width, depth = self.observation_space.shape
        top_margin = img_height // top_margin_divider
        img_height = img_height - top_margin
        # Region Of Interest
        # r = [margin_left, margin_top, width, height]
        self.roi = [0, top_margin, img_width, img_height]

        self.observation_space = spaces.Box(
            self.observation_space.low[0, 0, 0],
            self.observation_space.high[0, 0, 0],
            (img_height, img_width, depth),
            dtype=self.observation_space.dtype)

    def observation(self, observation):
        r = self.roi
        observation = observation[int(r[1]):int(r[1] + r[3]), int(r[0]):int(r[0] + r[2])]
        return observation

In [ ]:
class ResizeWrapper(gym.ObservationWrapper):
    def __init__(self, env=None, shape=(120, 160)):
        super(ResizeWrapper, self).__init__(env)
        if isinstance(shape, str):
            self.shape = eval(shape) + (self.observation_space.shape[2],)  # Depth is unchanged and can have any value
        else:
            self.shape = shape + (self.observation_space.shape[2],)  # Depth is unchanged and can have any value
        self.observation_space = spaces.Box(
            self.observation_space.low[0, 0, 0],
            self.observation_space.high[0, 0, 0],
            self.shape,
            dtype=self.observation_space.dtype)

    def observation(self, observation):
        resized = cv2.resize(observation, self.shape[:2][::-1], interpolation=cv2.INTER_AREA, )
        if len(resized.shape) == 2:
            resized = np.expand_dims(resized, 2)
        return resized

In [ ]:
class MotionBlurWrapper(gym.ObservationWrapper):
    """
    Simulates motion blur separately for horizontal (left-right) rotational and forward movement.
    Both are simulated using a smoothing convolutional filter (with varying sized "horizontal" or "vertical" kernel).
    Forward motion should produce non-uniform blur (none at the center point of the horizon, and larger towards the
     edges), here it is simulated using the same filtering over the whole image (the size of the kernel depends on the
     robot velocity.

    According to gluPerspective (used by the Simulator) x is width, y is height.
    """

    def __init__(self, env):
        super(MotionBlurWrapper, self).__init__(env)
        self.camera_fov_height_angle = self.unwrapped.cam_fov_y / 180. * np.pi
        self.camera_fov_width_angle = \
            self.unwrapped.camera_width / float(self.unwrapped.camera_height) * self.camera_fov_height_angle
        self.camera_height_px = self.observation_space.shape[0]
        self.camera_width_px = self.observation_space.shape[1]
        self.blur_time = 0.05
        self.prev_angle = self.unwrapped.cur_angle
        self.simulate_rotational_blur = True
        self.simulate_forward_blur = False  # Vertical convolution doesn't work well for this

    def observation(self, observation):
        if self.simulate_rotational_blur:
            cur_angle = self.unwrapped.cur_angle
            angular_vel = self._angle_diff(self.prev_angle,
                                           cur_angle) * self.unwrapped.frame_rate * self.unwrapped.frame_skip
            self.prev_angle = cur_angle
            delta_angle = angular_vel * self.blur_time
            if abs(delta_angle) > 0:
                ksize = np.round(np.abs(delta_angle / self.camera_fov_width_angle * self.camera_width_px)).astype(
                    int) + 1
                logger.debug("Rotational motion blur kernel size {}".format(ksize))
                kernel = np.zeros((ksize, ksize))
                kernel[ksize // 2, :] = 1. / ksize
                observation = cv2.filter2D(observation, -1, kernel)
        if self.simulate_forward_blur:
            # Empirical kernel size, proportional to the current speed over the max speed of the robot
            ksize = np.round(
                np.abs(self.camera_width_px / 30 * self.unwrapped.speed / self.unwrapped.robot_speed)).astype(int) + 1
            logger.debug("Forward motion blur kernel size {}".format(ksize))
            kernel = np.zeros((ksize, ksize))
            kernel[:, ksize // 2] = 1. / ksize
            observation = cv2.filter2D(observation, -1, kernel)
        return observation

In [ ]:
class NormalizeWrapper(gym.ObservationWrapper):
    def __init__(self, env=None):
        super(NormalizeWrapper, self).__init__(env)
        self.obs_lo = self.observation_space.low[0, 0, 0]
        self.obs_hi = self.observation_space.high[0, 0, 0]
        obs_shape = self.observation_space.shape
        self.observation_space = spaces.Box(0.0, 1.0, obs_shape, dtype=np.float32)

    def observation(self, obs):
        if self.obs_lo == 0.0 and self.obs_hi == 1.0:
            return obs
        else:
            return (obs - self.obs_lo) / (self.obs_hi - self.obs_lo)

In [ ]:
logger = logging.getLogger(__name__)


class Environment:
    def __init__(self, env_config, seed):
        self._env_config = env_config
        self._env = None
        np.random.seed(seed)
        random.seed(seed)

    def create_env(self, env_config, env_id=0) -> Simulator:
        self._env = Simulator(
            seed=self._env_config["seed"],
            map_name=self._env_config["map_name"],
            max_steps=self._env_config["max_steps"],
            camera_width=self._env_config["camera_width"],
            camera_height=self._env_config["camera_height"],
            accept_start_angle_deg=self._env_config["accept_start_angle_deg"],
            full_transparency=self._env_config["full_transparency"],
            distortion=self._env_config["distortion"],
            domain_rand=self._env_config["domain_rand"],
        )
        self._wrap()
        return self._env

    def _wrap(self) -> None:
        #self._env = LastPictureObsWrapper(self._env)
        #self._env = ActionDelayWrapper(self._env)
        self._env = ClipImageWrapper(self._env, 3)
        #self._env = RGB2GrayscaleWrapper(self._env)
        self._env = ResizeWrapper(self._env, (84, 84))
        #self._env = RandomFrameRepeatingWrapper(self._env)
        #self._env = ObservationBufferWrapper(self._env)
        self._env = MotionBlurWrapper(self._env)
        self._env = NormalizeWrapper(self._env)
        self._env = Heading2WheelVelsWrapper(self._env)
        #self._env = ActionSmoothingWrapper(self._env)
        #self._env = DtRewardTargetOrientation(self._env)
        #self._env = DtRewardVelocity(self._env)
        #self._env = DtRewardCollisionAvoidance(self._env)


In [ ]:
ray.shutdown()

from pyvirtualdisplay import Display
import matplotlib.pyplot as plt
%matplotlib inline

import os
def create_display():
    display = Display(visible=False, size=(1400, 900))
    d = display.start()
    if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
        !bash ../xvfb start
        %env DISPLAY=:0
    return d

display = create_display()

!Xvfb :0 -screen 0 1024x768x24 -ac +extension GLX +render -noreset &> xvfb.log &
!export DISPLAY=:0

ENV_NAME = 'Duckietown'
ray_init_config = {
    "num_cpus": 16,
    "num_gpus": 1,
}
config = {
    "seed": random.randint(0, 100000),
    "map_name": "loop_empty",
    "max_steps": 5000,
    "camera_width": 640,
    "camera_height": 480,
    "accept_start_angle_deg": 40,
    "full_transparency": True,
    "distortion": True,
    "domain_rand": False
}
env = Environment(config, random.randint(0, 100000))

ray.init(**ray_init_config)

register_env(ENV_NAME, env.create_env)

rllib_config = get_default_rllib_conf()
rllib_config.update({
    "env": ENV_NAME,
    "num_gpus": 1,
    "num_workers": 15,  # 32,
    "gpu_per_worker": 1,
    "env_per_worker": 2,
    "framework": "torch",
    "lr": 0.0001,
})

conf = update_conf(rllib_config)

tune.run(
    PPOTrainer,
    stop={"timesteps_total": 2000000},
    checkpoint_at_end=True,
    config=conf,
    trial_name_creator=lambda trial: trial.trainable_name,
    checkpoint_freq=1,
    keep_checkpoints_num=2,
    checkpoint_score_attr="episode_reward_mean"
)

2022-03-08 12:23:29,613	INFO services.py:1376 -- View the Ray dashboard at http://127.0.0.1:8267


Trial name,status,loc
PPOTrainer,PENDING,


(bundle_reservation_check_func pid=7247) 
(PPOTrainer pid=7247) DEBUG:commons:version: 6.2.4 *
(PPOTrainer pid=7247) DEBUG:typing:version: 6.2.3
(PPOTrainer pid=7247) DEBUG:duckietown_world:duckietown-world version 6.2.38 path /usr/local/lib/python3.7/dist-packages
(PPOTrainer pid=7247) DEBUG:geometry:PyGeometry-z6 version 2.1.4 path /usr/local/lib/python3.7/dist-packages
(PPOTrainer pid=7247) DEBUG:aido_schemas:aido-protocols version 6.0.59 path /usr/local/lib/python3.7/dist-packages
(PPOTrainer pid=7247) DEBUG:nodes:version 6.2.13 path /usr/local/lib/python3.7/dist-packages pyparsing 3.0.7


(PPOTrainer pid=7247) {'audio': ('directsound', 'openal', 'pulse', 'silent'), 'debug_font': False, 'debug_gl': True, 'debug_gl_trace': False, 'debug_gl_trace_args': False, 'debug_graphics_batch': False, 'debug_lib': False, 'debug_media': False, 'debug_texture': False, 'debug_trace': False, 'debug_trace_args': False, 'debug_trace_depth': 1, 'debug_trace_flush': True, 'debug_win32': False, 'debug_x11': False, 'graphics_vbo': True, 'shadow_window': True, 'vsync': None, 'xsync': True, 'xlib_fullscreen_override_redirect': False, 'darwin_cocoa': True, 'search_local_libs': True}


(PPOTrainer pid=7247) DEBUG:gym-duckietown:gym-duckietown version 6.1.31 path /usr/local/lib/python3.7/dist-packages
(PPOTrainer pid=7247) 
(PPOTrainer pid=7247) E0308 12:23:46.085388309    7247 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(PPOTrainer pid=7247) E0308 12:23:46.113088155    7247 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(PPOTrainer pid=7247) E0308 12:23:46.150228940    7247 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(PPOTrainer pid=7247) E0308 12:23:46.279833223    7247 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(PPOTrainer pid=7247) E0308 12:23:46.308960760    7247 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(PPOTrainer pid=7247) E0308 12:23:46.404930042    7247 fork_posix

...

(RolloutWorker pid=7717) {'audio': ('directsound', 'openal', 'pulse', 'silent'), 'debug_font': False, 'debug_gl': True, 'debug_gl_trace': False, 'debug_gl_trace_args': False, 'debug_graphics_batch': False, 'debug_lib': False, 'debug_media': False, 'debug_texture': False, 'debug_trace': False, 'debug_trace_args': False, 'debug_trace_depth': 1, 'debug_trace_flush': True, 'debug_win32': False, 'debug_x11': False, 'graphics_vbo': True, 'shadow_window': True, 'vsync': None, 'xsync': True, 'xlib_fullscreen_override_redirect': False, 'darwin_cocoa': True, 'search_local_libs': True}


(RolloutWorker pid=7659) E0308 12:27:17.176429908    7659 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(RolloutWorker pid=7481) DEBUG:gym-duckietown:loading mesh 'duckie' from file_path '/usr/local/lib/python3.7/dist-packages/duckietown_world/data/gd1/meshes/duckie/duckie.obj'
(RolloutWorker pid=7659) E0308 12:27:17.444379688    7659 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(RolloutWorker pid=7659) E0308 12:27:17.897654020    7659 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(RolloutWorker pid=7491) DEBUG:gym-duckietown:loading mesh 'cone' from file_path '/usr/local/lib/python3.7/dist-packages/duckietown_world/data/gd1/meshes/cone/cone.obj'
(RolloutWorker pid=7659) E0308 12:27:18.199220246    7659 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


(RolloutWorker pid=7689) {'audio': ('directsound', 'openal', 'pulse', 'silent'), 'debug_font': False, 'debug_gl': True, 'debug_gl_trace': False, 'debug_gl_trace_args': False, 'debug_graphics_batch': False, 'debug_lib': False, 'debug_media': False, 'debug_texture': False, 'debug_trace': False, 'debug_trace_args': False, 'debug_trace_depth': 1, 'debug_trace_flush': True, 'debug_win32': False, 'debug_x11': False, 'graphics_vbo': True, 'shadow_window': True, 'vsync': None, 'xsync': True, 'xlib_fullscreen_override_redirect': False, 'darwin_cocoa': True, 'search_local_libs': True}


(RolloutWorker pid=7659) E0308 12:27:18.435245868    7659 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(RolloutWorker pid=7491) DEBUG:gym-duckietown:loading texture: /usr/local/lib/python3.7/dist-packages/duckietown_world/data/gd1/meshes/cone/cone.png
(RolloutWorker pid=7481) DEBUG:gym-duckietown:loading mesh 'cone' from file_path '/usr/local/lib/python3.7/dist-packages/duckietown_world/data/gd1/meshes/cone/cone.obj'
(RolloutWorker pid=7659) E0308 12:27:18.884858441    7659 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
(RolloutWorker pid=7536) DEBUG:gym-duckietown:loading texture: /usr/local/lib/python3.7/dist-packages/duckietown_world/data/gd1/meshes/duckiebot/circlegrid-square.jpg
(RolloutWorker pid=7481) DEBUG:gym-duckietown:loading texture: /usr/local/lib/python3.7/dist-packages/duckietown_world/data/gd1/meshes/cone/cone.png
(RolloutWorker pid=7491) DEBUG:gym-duckiet

KeyboardInterrupt: ignored

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

